In [1]:
import sys
import os

#####################
# Import of utils.py functions
#####################
root_dir = os.getcwd()
sys.path.append('.')
from utils import loadFSL, FSLeyesServer

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################

# ???
# os.environ["DIPY_HOME"] = "/home/jovyan/Data"

#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries
##################
import fsl.wrappers
from fsl.wrappers import fslmaths, flirt, fast, bet
from fsl.wrappers.misc import fslroi
import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri
import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

import pandas as pd
import numpy as np
import json
import subprocess

In [2]:
# Cleanup previously defined data
os.system("rm -rf dataset/derivatives")

0

In [3]:
%gui wx

Gtk-Message: 10:51:00.951: Failed to load module "canberra-gtk-module"


In [4]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

10:51:06: Debug: Adding duplicate image handler for 'Windows bitmap file'
10:51:06: Debug: Adding duplicate animation handler for '1' type
10:51:06: Debug: Adding duplicate animation handler for '2' type
10:51:06: Debug: Adding duplicate image handler for 'Windows bitmap file'
10:51:06: Debug: Adding duplicate animation handler for '1' type
10:51:06: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:15137): Gtk-CRITICAL **: 10:51:06.295: gtk_window_resize: assertion 'height > 0' failed


In [5]:
# Creating derivative root and saving some useful paths
subject = "subject101410"

bids_root = op.join(root_dir, "dataset")
subject_root = op.join(bids_root, subject)

derivatives_root = op.join(bids_root, "derivatives")
os.makedirs(derivatives_root, exist_ok=True)
preproc_root = op.join(derivatives_root, "preprocessed_data")
os.makedirs(preproc_root, exist_ok=True)
subject_preproc_root = op.join(preproc_root, subject)
os.makedirs(subject_preproc_root, exist_ok=True)

anat_path = op.join(subject_root, "T1w", "T1w.nii.gz")

In [6]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(anat_path)


(ipykernel_launcher.py:15137): Gdk-WARNING **: 10:52:05.108: gdkdrawable-x11.c:952 drawable is not a pixmap or window


In [7]:
# To display the commands and how to use the "bet" command
os.system("bet -h")


Usage:    bet <input> <output> [options]

Main bet2 options:
  -o          generate brain surface outline overlaid onto original image
  -m          generate binary brain mask
  -s          generate approximate skull image
  -n          don't generate segmented brain image output
  -f <f>      fractional intensity threshold (0->1); default=0.5; smaller values give larger brain outline estimates
  -g <g>      vertical gradient in fractional intensity threshold (-1->1); default=0; positive values give larger brain outline at bottom, smaller at top
  -r <r>      head radius (mm not voxels); initial surface sphere is set to half of this
  -c <x y z>  centre-of-gravity (voxels not mm) of initial mesh surface.
  -t          apply thresholding to segmented brain image and mask
  -e          generates brain surface as mesh in .vtk format

Variations on default bet2 functionality (mutually exclusive options):
  (default)   just run bet2
  -R          robust brain centre estimation (iterates BE

256

In [8]:
# Skull stripping with "bet"
robust: bool = True

# Define the path where to save the file
os.makedirs(op.join(subject_preproc_root, "T1w"), exist_ok=True)
betted_brain_path = op.join(subject_preproc_root, "T1w", "T1w_skull.nii.gz")

# Apply "bet" and save the file
robust_text = "-R" if robust else ""
os.system(f"bet {anat_path} {betted_brain_path} -m {robust_text}")

# BET also creates a resulting Mask that is useful to display
resulting_mask = op.join(subject_preproc_root, "T1w", "T1w_skull_mask.nii.gz")

print("Skull stripping with BET succeeded!")

Skull stripping with BET succeeded!


In [9]:
# Display the Skull Stripped Image
fsleyesDisplay.load(resulting_mask)

In [10]:
# Now we have to apply the mask to the image
def apply_fsl_math_approach(img_path, mask_path, masked_img_path):
    os.system(f'fslmaths {img_path} -mas {mask_path} {masked_img_path}')

apply_fsl_math_approach(anat_path, resulting_mask, betted_brain_path)

In [11]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)

In [12]:
os.system("fast -h")


Part of FSL (ID: "")
FAST 
Copyright(c) 2004-2012, University of Oxford

Usage: 
fast [options] file(s)

Optional arguments (You may optionally specify one or more of):
	-n,--class	number of tissue-type classes; default=3
	-I,--iter	number of main-loop iterations during bias-field removal; default=4
	-l,--lowpass	bias field smoothing extent (FWHM) in mm; default=20
	-t,--type	type of image 1=T1, 2=T2, 3=PD; default=T1
	-f,--fHard	initial segmentation spatial smoothness (during bias field estimation); default=0.02
	-g,--segments	outputs a separate binary image for each tissue type
	-a <standard2input.mat> initialise using priors; you must supply a FLIRT transform
	-A <prior1> <prior2> <prior3>    alternative prior images
	--nopve	turn off PVE (partial volume estimation)
	-b		output estimated bias field
	-B		output bias-corrected image
	-N,--nobias	do not remove bias field
	-S,--channels	number of input images (channels); default 1
	-o,--out	output basename
	-P,--Prior	use priors throug

256

In [13]:
# tissue segmentation
fast_output_path = op.join(subject_preproc_root, "T1w", "T1w_fasted")
fast_target = betted_brain_path 

# Quick cleanup
[os.remove(f) for f in glob.glob(op.join(subject_preproc_root, "T1w", '*fast*'))]

fast(imgs=[fast_target], out=fast_output_path, n_classes=3)

{}

In [15]:
print_dir_tree(preproc_root, max_depth=3)

|preprocessed_data/
|--- subject101410/
|------ T1w/
|--------- T1w_fasted_mixeltype.nii.gz
|--------- T1w_fasted_pve_0.nii.gz
|--------- T1w_fasted_pve_1.nii.gz
|--------- T1w_fasted_pve_2.nii.gz
|--------- T1w_fasted_pveseg.nii.gz
|--------- T1w_fasted_seg.nii.gz
|--------- T1w_skull.nii.gz
|--------- T1w_skull_mask.nii.gz


In [16]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)
fsleyesDisplay.load(glob.glob(op.join(subject_preproc_root, 'T1w','*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(subject_preproc_root, 'T1w','*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(subject_preproc_root, 'T1w','*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

## I have checked and made sure everything works up to here, Check what is going on later

In [ ]:
# checking what does RL and LR mean

func_LR_path = op.join(subject_root, 'fMRI', 'tfMRI_MOTOR_LR','tfMRI_MOTOR_LR.nii')
func_RL_path = op.join(subject_root, 'fMRI', 'tfMRI_MOTOR_RL','tfMRI_MOTOR_RL.nii')

# fsleyesDisplay.resetOverlays()
# fsleyesDisplay.load(func_LR_path)
# fsleyesDisplay.load(func_RL_path)
# same reference

In [ ]:
# scaling variance to one
import numpy as np
temp = nib.load(func_LR_path)
LR_func_data= temp.get_fdata()
std_LR=LR_func_data.std(axis=(0,1,2))
LR_norm = LR_func_data / std_LR
LR_norm_img = nib.Nifti1Image(LR_norm.astype(np.uint8), temp.affine, temp.header)
# LR_path = op.join(preproc_root, 'fMRI','tfMRI_MOTOR_LR_norm.nii')
func_preproc_path = op.join(preproc_root, 'fMRI')
os.mkdirs(func_preproc_path, exists_ok=True)
nib.save(LR_norm_img, op.join(func_preproc_path,'tfMRI_MOTOR_LR_norm.nii'))

temp = nib.load(func_RL_path)
RL_func_data= temp.get_fdata()
std_RL=RL_func_data.std(axis=(0,1,2))
RL_norm = RL_func_data / std_RL
RL_norm_img = nib.Nifti1Image(RL_norm.astype(np.uint8), temp.affine, temp.header)
# RL_path = op.join(preproc_root, 'fMRI','tfMRI_MOTOR_RL_norm.nii')
nib.save(RL_norm_img, op.join(func_preproc_path,'tfMRI_MOTOR_RL_norm.nii'))

In [35]:
# concatenating runs
print(LR_norm.std())

1.000000021161999


In [22]:
subprocess.run(['fslmerge', '-t', merged_phase_imgs, 
                    op.join(fmap_path, 'sub-001_acq-task_dir-AP_epi.nii.gz'), 
                    op.join(fmap_path, 'sub-001_acq-task_dir-PA_epi.nii.gz')])

sh: 1: merge: not found


32512